In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC  # Support Vector Classifier for SVM
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier  # Assuming RRF akin to ExtraTrees
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Function to load and prepare the dataset from a CSV file
def load_and_prepare_data(train):
    df = pd.read_csv(train)
    X = df.drop('Outcome', axis=1)  # Features
    y = df['Outcome']  # Target variable

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Feature Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

In [ ]:
# Define classifiers
def define_classifiers():
    lda = LDA()
    svm = SVC(kernel='linear', probability=True)  # Linear kernel used for SVM
    knn = KNN(n_neighbors=5)
    rf = RandomForestClassifier(n_estimators=100)
    rrf = ExtraTreesClassifier(n_estimators=100)  # Using ExtraTrees as a stand-in for RRF

    classifiers = {
        'LDA': lda,
        'SVM': svm,
        'KNN': knn,
        'RF': rf,
        'RRF': rrf
    }

    return classifiers

In [ ]:
# Perform cross-validation and evaluate classifiers
def evaluate_classifiers(X, y, classifiers):
    results = {}
    for name, clf in classifiers.items():
        scores = cross_val_score(clf, X, y, cv=5)  # 5-fold cross-validation
        results[name] = np.mean(scores), np.std(scores)  # Store mean accuracy and standard deviation

    return results

In [ ]:
def load_and_prepare_data(csv_file):
    df = pd.read_csv(csv_file)
    print(df.columns)  # Add this line to check column names
    # Rest of your code to prepare data

In [ ]:
# Main function to orchestrate the process
def main(csv_file):
    X_train, X_test, y_train, y_test = load_and_prepare_data(csv_file)
    classifiers = define_classifiers()
    results = evaluate_classifiers(X_train, y_train, classifiers)

    for name, (mean_acc, std_dev) in results.items():
        print(f"{name}: Accuracy={mean_acc*100:.2f}%, StdDev={std_dev*100:.2f}%")

# Path to the CSV file (this needs to be updated with the actual path where the 'diabetes.csv' is stored)
csv_file_path = '/content/diabetes.csv'

# Uncomment the following line to run the program with the actual CSV file path
main(csv_file_path)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv('/content/diabetes.csv')

# Data preprocessing
# Handling missing values (if any)
data.fillna(data.mean(), inplace=True)

# Feature and target variables
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Data normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model selection
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
knn = KNeighborsClassifier()
ada = AdaBoostClassifier(random_state=42)
log_reg = LogisticRegression(random_state=42)

# Ensemble approach - Voting Classifier
ensemble = VotingClassifier(estimators=[
    ('decision_tree', dt),
    ('random_forest', rf),
    ('knn', knn),
    ('adaboost', ada),
    ('logistic_regression', log_reg)
], voting='hard')

# Training the ensemble model
ensemble.fit(X_train, y_train)

# Prediction and Evaluation
y_pred = ensemble.predict(X_test)
accuracy = accuracy_score(y_test, y_pred) * 100  # Convert to percentage
print(f'Ensemble model accuracy: {accuracy:.2f}%')

FileNotFoundError: [Errno 2] No such file or directory: '/content/diabetes.csv'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from google.colab import files
from IPython.display import Image as IPImage, display

# Function to load and preprocess the data
def load_and_preprocess_data(dataset_dir, img_height=256, img_width=256, batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2  # Splitting the dataset into 80% training and 20% validation
    )

    train_data = train_datagen.flow_from_directory(
        dataset_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',  # Change to 'categorical' for multi-class classification
        subset='training'
    )

    val_data = train_datagen.flow_from_directory(
        dataset_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_data, val_data

# Function to build a CNN model
def build_cnn_model(input_shape=(256, 256, 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(5, activation='softmax')  # Adjusted to 6 units for 6 classes
    ])
    return model

# Function to train and evaluate a model
def train_and_evaluate(model, train_data, val_data, epochs=10):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, validation_data=val_data, epochs=epochs)
    return history

# Function to process and predict the class of the input image
def predict_image_class(model, img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(256, 256))  # Adjust the size according to your model's input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create a batch
    img_array /= 255.0  # Rescale pixel values

    # Make predictions
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    # Map the predicted class index to the class name
    class_names = ['Healthy', 'Mild', 'Moderate', 'Proliferate', 'Severe', 'Other']  # Adjust according to your classes
    predicted_class_name = class_names[predicted_class[0]]

    return predicted_class_name

# Function to upload an image and predict its class
def upload_and_predict(model):
    uploaded = files.upload()

    for img_path in uploaded.keys():
        display(IPImage(img_path))  # Display the uploaded image
        predicted_class_name = predict_image_class(model, img_path)
        print(f"Predicted Class: {predicted_class_name}")

# Main execution function
def main():
    # Path where your dataset folders are located
    dataset_directory = "/content/drive/MyDrive/MP2"  # Update this path as needed

    # Path where you want to save the trained model
    model_save_path = "/content/final_model.h5"  # Update this path as needed

    train_data, val_data = load_and_preprocess_data(dataset_directory)

    cnn_model = build_cnn_model()

    print("Training CNN Model...")
    train_and_evaluate(cnn_model, train_data, val_data)

    # Save the trained model
    cnn_model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    print("Upload an image to predict DR...")
    upload_and_predict(cnn_model)

main()


Found 2200 images belonging to 5 classes.
Found 550 images belonging to 5 classes.
Training CNN Model...
Epoch 1/10
17/69 [======>.......................] - ETA: 11:04 - loss: 1.9718 - accuracy: 0.4504

In [ ]:
pip install gradio==3.50

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.0 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=f67ae7263e02b825d43a84ab448fd12668688ba42a143ad6d5cdb376ec00c920

In [ ]:
import gradio as gr
import numpy as np
from PIL import Image

# Function to process and predict the class of the input image
def predict_image_class(model, img):
    # Resize the image to match the model's input shape
    img_resized = img.resize((256, 256))

    # Convert the image to numpy array
    img_array = np.array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)  # Create a batch
    img_array = img_array / 255.0  # Rescale pixel values

    # Make predictions
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    # Map the predicted class index to the class name
    class_names = ['Healthy', 'Mild', 'Moderate', 'Proliferate', 'Severe', 'Other']
    predicted_class_name = class_names[predicted_class[0]]

    return predicted_class_name

# Load the trained model
model = tf.keras.models.load_model("/content/final_model.h5")

# Define the Gradio interface
image_input = gr.inputs.Image()
output = gr.outputs.Label()

# Create the Gradio interface
gr.Interface(fn=predict_image_class, inputs=image_input, outputs=output, model=model, capture_session=True).launch(share=True)


<ipython-input-11-2a4c57911c1f>:29: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image_input = gr.inputs.Image()
<ipython-input-11-2a4c57911c1f>:29: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image_input = gr.inputs.Image()
<ipython-input-11-2a4c57911c1f>:30: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output = gr.outputs.Label()
<ipython-input-11-2a4c57911c1f>:30: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  output = gr.outputs.Label()
<ipython-input-11-2a4c57911c1f>:33: GradioDeprecationWarning: `capture_session` parameter is deprecated, and it has no effect
  gr.Interface(fn=predict_image_class, inputs=image_input, outputs=output, model=model, capture_s

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ef9fc33984e5a219f4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import cv2  # For image processing
import numpy as np  # For numerical operations
from skimage import exposure  # For adaptive histogram equalization
import pywt  # For discrete wavelet transform
from sklearn.svm import SVC  # For Support Vector Machine classifier
from sklearn.metrics import accuracy_score  # For calculating accuracy
# Add other necessary imports here (e.g., for fuzzy C-means segmentation)

In [ ]:
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def adaptive_histogram_equalization(image):
    return exposure.equalize_adapthist(image, clip_limit=0.03)

def discrete_wavelet_transform(image):
    coeffs = pywt.dwt2(image, 'haar')
    cA, (cH, cV, cD) = coeffs
    return cA

def apply_matched_filter(image):
    # This function would apply a matched filter to the image.
    # Implementation would depend on the specific filter and requirements.
    pass

def fuzzy_c_means_segmentation(image):
    # This function would apply fuzzy C-means segmentation to the image.
    # Implementation depends on the choice of library or custom implementation.
    pass

In [ ]:
def extract_features(image):
    # Apply the preprocessing functions
    grayscale = convert_to_grayscale(image)
    equalized = adaptive_histogram_equalization(grayscale)
    wavelet_transformed = discrete_wavelet_transform(equalized)
    # Further steps would involve extracting specific features from the preprocessed image,
    # potentially using matched filter response and fuzzy C-means as part of the process.
    # The exact implementation would depend on the specific features being extracted.
    features = []  # This would be replaced by actual feature extraction logic
    return features

In [ ]:
def load_and_preprocess_dataset(dataset_path):
    # This function would load images from the dataset, preprocess them,
    # and extract features. The exact implementation would depend on the
    # dataset's structure and the specific preprocessing steps.
    pass

In [ ]:
load_and_preprocess_dataset(dataset_path)

NameError: name 'dataset_path' is not defined

In [ ]:
def train_svm_classifier(features, labels):
    svm_classifier = SVC(kernel='linear')  # Linear kernel is used as an example
    svm_classifier.fit(features, labels)
    return svm_classifier

In [ ]:
def main():
    dataset_path = '/content/diabetes.csv'
    features, labels = load_and_preprocess_dataset(dataset_path)
    svm_classifier = train_svm_classifier(features, labels)

    # To evaluate the classifier, you would need a test set.
    # You would preprocess the test set in the same way as the training set,
    # then use the classifier to make predictions and compare them to the true labels.
    test_features, test_labels = load_and_preprocess_dataset('path/to/test_set')
    predictions = svm_classifier.predict(test_features)
    accuracy = accuracy_score(test_labels, predictions)
    print(f'Accuracy: {accuracy * 100:.2f}%')

if __name__ == '__main__':
    main()

TypeError: cannot unpack non-iterable NoneType object